<a href="https://colab.research.google.com/github/RamanRed/story/blob/main/prompting_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-_7_hokhk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-_7_hokhk
  Resolved https://github.com/huggingface/transformers.git to commit 27903de7ecfc21e9b5a061c46c3b1ff73539d385
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9583114 sha256=8d8b0c3d2403255889bb8d7a442e4e9e2b767756578aac8758f16a5bbf80dd90
  Stored in directory: /tmp/pip-ephem-wheel-cache-tgvr3qzm/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Cloning https://github.com

In [ ]:
# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

In [ ]:
# loading the model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    load_in_4bit=True,                                                            # loading the model with 4 bit precision, so as to show the example on Colab GPU
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

PackageNotFoundError: No package metadata was found for bitsandbytes

In [ ]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant

In [ ]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256
)

# Zero Shot

In [ ]:
inputs = tokenizer("I'll bet the video game is alot more fun than the film.\Sentiment:", return_tensors="pt")
print(inputs)
input_ids = inputs["input_ids"].to('cuda')

{'input_ids': tensor([[    1,   306, 29915,   645,  1010,   278,  4863,  3748,   338,   394,
           327,   901,  2090,  1135,   278,  2706,  7790, 29903,   296,  2073,
         29901]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [ ]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

I'll bet the video game is alot more fun than the film.\Sentiment: 0. surely\
Too lazy to write a full review, but here are some quick thoughts on Sonic Forces:\nThe good stuff first - visually this game looks absolutely incredible in both handheld and docked modes (even better when you see it running at high framerates). The character models especially look amazing with all their animations captured perfectly down to even small details like blinking or moving around naturally while walking/running,\nNow for the not-so-good parts...the story mode feels very forced as if they were trying too hard making every cutscene feel cringe worthy without any real emotional weight behind them; characters come off feeling one dimensionally blandly personalityless most times unless voiced by someone talented outside of gaming who knows how deal with difficult dialogue mandated from above (\"Hey remember that thing we did back then? That was pretty cool yeah! *cough*"). Not only does no attempt go in

# Few Shot

In [ ]:
inputs = tokenizer('''
Text: (lawrence bounces) all over the stage, dancing, running, sweating, mopping his face and generally displaying the wacky talent that brought him fame in the first place.
Sentiment: positive

Text: despite all evidence to the contrary, this clunker has somehow managed to pose as an actual feature movie, the kind that charges full admission and gets hyped on
tv and purports to amuse small children and ostensible adults.
Sentiment: negative

Text: for the first time in years, de niro digs deep emotionally, perhaps because he's been stirred by the powerful work of his co-stars.
Sentiment: positive

Text: i'll bet the video game is a lot more fun than the film.
\Sentiment:''', return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

In [ ]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [ ]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

Text: (lawrence bounces) all over the stage, dancing, running, sweating, mopping his face and generally displaying the wacky talent that brought him fame in the first place.
Sentiment: positive

Text: despite all evidence to the contrary, this clunker has somehow managed to pose as an actual feature movie, the kind that charges full admission and gets hyped on
tv and purports to amuse small children and ostensible adults.
Sentiment: negative

Text: for the first time in years, de niro digs deep emotionally, perhaps because he's been stirred by the powerful work of his co-stars.
Sentiment: positive

Text: i'll bet the video game is a lot more fun than the film.
\Sentiment: neutral


# Instruct Prompting

In [ ]:
inputs = tokenizer('''Please label the sentiment towards the movie of the given movie review. The sentiment label should be "positive" or "negative".
Text: i'll bet the video game is a lot more fun than the film.
\Sentiment:''', return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

In [ ]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [ ]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

Please label the sentiment towards the movie of the given movie review. The sentiment label should be "positive" or "negative".
Text: i'll bet the video game is a lot more fun than the film.
\Sentiment: Positive
